# Music rearranger

End-to-end example notebook

In [ ]:
# Imports
import copy
import soundfile as sf
import librosa
import matplotlib.pyplot as plt
from rearranger.identification import *
from rearranger.formatting import *
from rearranger.segmentation import *
from rearranger.plotting import *
from rearranger.optimization import *
from rearranger.construction import *

In [ ]:
# Parameters
audio_path = ""
n_measures = 1  # min measure size for quantization
pattern_length = 2  # min diagonal length to consider a pattern
pattern_percentile = 50  # thresholds for connected point strength to consider
type_min_d_len = 2  # min diagonal length for inner-segment points
type_levels_list = [2, 4, 6]  # levels to check for inner-segment points
target_beats = 120  # set target number of beats for rearrangement here

In [ ]:
# Get segmentation data
beat_times, beat_order, A, A_f, fixed_levels, Csync, Msync, Hsync = stuff_I_need(audio_path)

In [ ]:
# Format and quantize structure representation
fixed_levels_beats = structure_time_to_beats(
    fixed_levels=fixed_levels,
    beat_times=beat_times)
fixed_levels_n_measures, downbeat_times, downbeat_beats, n_measure_beats = quantize_to_measures(
    fixed_levels_beats=fixed_levels_beats,
    n_measures=n_measures,
    beat_order=beat_order,
    beat_times=beat_times)
fixed_levels_n_measures_unique = get_unique_segments(fixed_levels_beats)

# Get patterns
patterns = common_patterns(
    Csync=Csync,
    Msync=Msync,
    Hsync=Hsync,
    length=pattern_length,
    percentile=pattern_percentile)

# Get cross-segment points
segment_points = segment_transition_points(
    fixed_levels=fixed_levels_n_measures_unique,
    quantization=n_measures,
    beats_in_measure=int(np.max(beat_order[:, 1])),
    patterns=patterns)

# Get inner-segment points
type_points = type_transition_points(
    fixed_levels=fixed_levels_n_measures,
    levels_list=type_levels_list,
    min_d_len=type_min_d_len,
    patterns=patterns,
    beats_in_measure=int(np.max(beat_order[:, 1])))

In [ ]:
# Path finding
jumps = get_ordinal_encoding(segment_points+type_points,
                             patterns.shape[0],
                             neighbors=True)
jumps_copy = copy.deepcopy(jumps)

result = get_path(
    cur_idx=0,
    jumps=jumps_copy,
    rem_beats=target_beats,
    beat_no=patterns.shape[0]
)

In [ ]:
# Construct audio
y, sr = librosa.load(audio_path)
y_new = audio_constructor(
    y=y,
    sr=sr,
    recon_beats=result,
    beat_times=beat_times,
    crossfade=0.1
)

sf.write(audio_path[:-4]+f'{target_beats}.wav', y_new, sr)